In [1]:
import SciServer
from SciServer import SkyServer
import matplotlib.pyplot as plt
from astropy.io import ascii, fits
from PIL import Image
from multiprocessing.pool import Pool
import itertools
import time
import tqdm
import numpy as np
import glob

/home/fortson/manth145/.conda/envs/tf_latest/lib/python3.9/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [3]:
def download_image(info):
    info_type ='ra_dec'
    obj_info_search_DR = "DR12" # Data release to be used to query the ObjID, PetroR90 information
    image_data_release = "DR12" 
    
    try:
        
        if info_type == 'ra_dec':
            ra, dec = info #unpack the RA and DEC info

            object_search_data =  SciServer.SkyServer.objectSearch(ra= ra, dec = dec, dataRelease=obj_info_search_DR) # Query the object information
            object_id = object_search_data[0]['Rows'][0]['objId'] # grab the ObjID 
            sql_query = "SELECT p.ra, p.dec, p.ObjID, p.PetroR90_r from PhotoPrimary p WHERE p.objID = %s"%(object_id) # set up SQL query to retrieve PetroR50_r
            queried_data = SciServer.SkyServer.sqlSearch(sql_query, dataRelease=obj_info_search_DR) # queried data

            petroR90 = queried_data['PetroR90_r'].values.item() # grab the PetroR90

    #         print('Petro R90 = %s'%(petroR90))
    #         print('Arcsec/pix = %s'%(petroR90 * 0.02)) # Scale to be used for cutouts, based on details from Willet et al.,

            required_side_length =  round(75/(petroR90 * 0.02)) #424 # Pixels per side; use round(150/(petroR90 * 0.02)) if in case you want 150 arcsec per side.
            # print('Arcsec per side = %s'%(side_length * petroR90 * 0.02))
    #         print('Required side length [pix] = %s'%required_side_length)
        elif info_type =='ID':
            sql_query = "SELECT p.ra, p.dec, p.ObjID, p.PetroR90_r from PhotoPrimary p WHERE p.objID = %s" %(info) # If we have ObjIDs, directly setup SQL query.
            queried_data = SciServer.SkyServer.sqlSearch(sql_query, dataRelease=image_data_release) # query from the specified DR above
            print(queried_data)
            petroR90 = queried_data['PetroR90_r'].values.item() # retrive RA, DEC, and PetroR90
            ra = queried_data['ra'].values.item()
            dec = queried_data['dec'].values.item()
            print('Petro R90 = %s' % (petroR90))
            print('Arcsec/pix = %s' % (petroR90 * 0.1))

            required_side_length = 424 #round(150/(petroR90 * 0.02)); same comment as above
            print('Required side length [pix] = %s' % required_side_length)


        img = SkyServer.getJpegImgCutout(ra = ra, dec = dec,
                                         scale = 0.02 *petroR90,
                                         width = required_side_length, height = required_side_length,
                                         dataRelease = image_data_release) # we have all the needed information, generate a cutout.

        im = Image.fromarray(img)
        im.save('/home/fortson/manth145/data/z20_candidates/%s_%s.jpg'%(object_id, image_data_release))
    except TimeoutError:
        pass
    return

In [4]:
# greenpeas = ascii.read('/home/fortson/manth145/data/GZ2-GAN/catalogs/Cardamone09_tab4.txt', format='no_header')

# pos = greenpeas[['col2', 'col3']]

# catalog = ascii.read('/home/fortson/manth145/data/GZ2-GAN/catalogs/Cardamone09_tab4.txt')

catalog = ascii.read('/home/fortson/manth145/data/catalogs/Coordinates_z20candidates.csv')


In [5]:
info = np.array(catalog[['RA', 'DEC']])

In [6]:
pl = Pool(processes=5)
result = list(tqdm.tqdm(pl.imap(download_image,  list(info),chunksize=1), total=len(info), ascii=True))

  0%|          | 0/1585 [00:00<?, ?it/s]/home/fortson/manth145/.conda/envs/tf_latest/lib/python3.9/site-packages/SciServer-2.1.0-py3.9.egg/SciServer/SkyServer.py:331: Warning: In Authentication.getToken: Authentication token is not defined: the user did not log in with the Authentication.login function, or the token has not been stored in the command line argument --ident.
/home/fortson/manth145/.conda/envs/tf_latest/lib/python3.9/site-packages/SciServer-2.1.0-py3.9.egg/SciServer/SkyServer.py:331: Warning: In Authentication.getToken: Authentication token is not defined: the user did not log in with the Authentication.login function, or the token has not been stored in the command line argument --ident.
/home/fortson/manth145/.conda/envs/tf_latest/lib/python3.9/site-packages/SciServer-2.1.0-py3.9.egg/SciServer/SkyServer.py:331: Warning: In Authentication.getToken: Authentication token is not defined: the user did not log in with the Authentication.login function, or the token has not be

ConnectionError: None: Max retries exceeded with url: /DR12/SkyServerWS/SearchTools/SqlSearch?format=csv&cmd=SELECT%20p.ra,%20p.dec,%20p.ObjID,%20p.PetroR90_r%20from%20PhotoPrimary%20p%20WHERE%20p.objID%20=%200x112d0b0560bb0251&TaskName=SciScript-Python.SkyServer.sqlSearch& (Caused by None)

In [16]:
list(info)

[587730774425665700,
 587727220876640877,
 587727180060098742,
 588015509806252142,
 587731187277627683,
 587731186203951111,
 587730775499407505,
 587730774962536621,
 587730775499407504,
 587730773888794648,
 587727225690128558,
 587727222487318704,
 587727220876705929,
 587727225690128563,
 587727220876705935,
 587731187814563978,
 587730773351923943,
 587727177912615045,
 587730775499407560,
 587727179523227783]